# transfomer를 사용한 Kobart 모델 파인 튜닝

## 1.  사전 작업

In [2]:
# 라이브러리 임포트

import json
import nltk
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.dataset as ds
import torch
from datasets import Dataset
from transformers import BartForConditionalGeneration
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import PreTrainedTokenizerFast
import evaluate

In [3]:
#사전 학습된 모델 가져오기
#Hugging face에서 자동적으로 가져옴
tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v2')
model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-base-v2')

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [4]:
#평가 함수 정의
def compute_metrics(eval_pred):
    """
    모델  평가 지표 계산 함수
    rouge score 퍼센트 반환
    """

    metric = evaluate.load("rouge")

    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [5]:
## 데이터 전처리 함수 한줄 한줄씩 되어 있는 리스트를 통합하는 전처리 함수
def extract_body(article) -> str:

    art_sentence = []

    for contents in article:
        if len(contents) >= 2:
            for sub_con in contents:
                art_sentence.append(sub_con['sentence'])
                continue
        elif len(contents) == 0:
            pass
        else:
            art_sentence.append(contents[0]['sentence'])

    return art_sentence

In [6]:
# 문자들 토큰화 함수
def preprocess_function(examples):
    prefix = ''
    max_input_length = 2000
    max_target_length = 750
    inputs = [prefix + doc for doc in examples["body"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    # 데이터가 손실될 경우에 앞의 단어가 아니라 뒤의 단어가 삭제되도록 하고싶다면 truncating이라는 인자를 사용합니다. truncating='post'를 사용할 경우 뒤의 단어가 삭제됩니다.
    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["abstractive"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [7]:
# 문장의 태그 같은 불필요 요소 제거 함수
def sentence_validation(art_sentence):
    del_sentence = []
    for sentence in art_sentence:
        if '@' in sentence or '/사진' in sentence:
            del_sentence.append(sentence)
        elif sentence[-1] != '.':
            del_sentence.append(sentence)
    for del_sen in del_sentence:
        art_sentence.remove(del_sen)
    return ' '.join(art_sentence)

## 2. 쿠다 환경 확인

In [8]:
torch.cuda.is_available()


True

## 3. 데이터 읽기

In [9]:
file_location = '../data/'

In [10]:

with open(f'{file_location}train_original.json', encoding='utf-8') as train_f:
    train_data = json.loads(train_f.read())
    train_df = pd.DataFrame(train_data['documents'])

with open(f'{file_location}valid_original.json', encoding='utf-8') as valid_f:
    valid_data = json.loads(valid_f.read())
    valid_df = pd.DataFrame(valid_data['documents'])

## 4. 데이터 전처리

In [11]:
train_df['body'] = train_df.text.apply(lambda x: sentence_validation(extract_body(x)))
valid_df['body'] = valid_df.text.apply(lambda x: sentence_validation(extract_body(x)))

train_df.abstractive = train_df.abstractive.apply(lambda x: x[0])
valid_df.abstractive = valid_df.abstractive.apply(lambda x: x[0])

train_df.drop(train_df.body.str.len().sort_values(ascending=False).head(1).index[0], inplace=True)

dataset = ds.dataset(pa.Table.from_pandas(train_df).to_batches())

### convert to Huggingface dataset
hg_dataset = Dataset(pa.Table.from_pandas(train_df))
# 테스트용으로 validation 데이터 1000개만 사용
hg_dataset_test = Dataset(pa.Table.from_pandas(valid_df[:1000]))


## 5. 데이터 토큰화

In [12]:
tokenized_data = hg_dataset.map(preprocess_function, batched=True)
tokenized_data_test = hg_dataset_test.map(preprocess_function, batched=True)


Map:   0%|          | 0/243982 [00:00<?, ? examples/s]

C:\Users\Chloe\miniconda3\envs\model\lib\site-packages\transformers\tokenization_utils_base.py:3596: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

## 6. 하이퍼 파라미터 튜닝

In [13]:
metric = evaluate.load("rouge")

batch_size = 2
model_name = 'kobart_v2'
args = Seq2SeqTrainingArguments(

    # 이 이름의 폴더를 생성하고 저장됨
    f"{model_name}-full",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=5,
    predict_with_generate=True,
    fp16=True,  # CUDA 설정을 완료한 후 사용가능.
    push_to_hub=False,
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

print('start training')
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_data,
    eval_dataset=tokenized_data_test,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)



start training


## 7.트레이닝 및 모델 저장

In [14]:
trainer.train()

C:\Users\Chloe\miniconda3\envs\model\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss


Exception ignored in: <function tqdm.__del__ at 0x0000023C545B31F0>
Traceback (most recent call last):
  File "C:\Users\Chloe\miniconda3\envs\model\lib\site-packages\tqdm\std.py", line 1145, in __del__
    self.close()
KeyboardInterrupt: 

KeyboardInterrupt



In [ ]:
trainer.save_model()